# CSV + API

In this reboot, we are going to use:

- The [Goodreads books](https://www.kaggle.com/jealousleopard/goodreadsbooks) dataset from Kaggle.
- The [Open Library Books API](https://openlibrary.org/dev/docs/api/books)

The goal of this livecode is to load the data from a CSV + loop over rows to enrich each row with information such as:

- List of subjects (Science, Humor, Travel, etc.)
- The cover URL of the book
- Other information you'd find useful in the JSON API

First, download the CSV in the local folder:

In [320]:
!curl -L https://gist.githubusercontent.com/ssaunier/351b17f5a7a009808b60aeacd1f4a036/raw/books.csv > books.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1509k  100 1509k    0     0  5167k      0 --:--:-- --:--:-- --:--:-- 5351k


In [321]:
import requests
import pandas as pd
import numpy as np

## Load books from CSV

In [329]:
books_df = pd.read_csv('books.csv')

In [330]:
books_df = books_df[['title', 'authors', 'isbn13']]

In [331]:
books_df

,title,authors,isbn13
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,9780439785969
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,9780439358071
2,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,9780439554930
3,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,9780439554893
4,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,9780439655484
...,...,...,...
13714,M Is for Magic,Neil Gaiman-Teddy Kristiansen,9780061186424
13715,Black Orchid,Neil Gaiman-Dave McKean,9780930289553
13716,InterWorld (InterWorld #1),Neil Gaiman-Michael Reaves,9780061238963
13717,The Faeries' Oracle,Brian Froud-Jessica Macbeth,9780743201117


In [332]:
books_df['cover_url'] = None

In [333]:
books_df

,title,authors,isbn13,cover_url
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,9780439785969,None
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,9780439358071,None
2,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,9780439554930,None
3,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,9780439554893,None
4,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,9780439655484,None
...,...,...,...,...
13714,M Is for Magic,Neil Gaiman-Teddy Kristiansen,9780061186424,None
13715,Black Orchid,Neil Gaiman-Dave McKean,9780930289553,None
13716,InterWorld (InterWorld #1),Neil Gaiman-Michael Reaves,9780061238963,None
13717,The Faeries' Oracle,Brian Froud-Jessica Macbeth,9780743201117,None


## API - Open Library

In [334]:
def fetch_book(isbn):
    url = "https://openlibrary.org/api/books?"
    
    params = {
        'bibkeys': f"ISBN:{isbn}",
        'format': 'json',
        'jscmd': 'data'
    }
    
    response = requests.get(url, params=params).json()
    
    if f"ISBN:{isbn}" in response:
        return response[f"ISBN:{isbn}"]
    return ""

In [343]:
fetch_book(9780439785969)

{'url': 'https://openlibrary.org/books/OL24280830M/Harry_Potter_and_the_Half-Blood_Prince',
 'key': '/books/OL24280830M',
 'title': 'Harry Potter and the Half-Blood Prince',
 'authors': [{'url': 'https://openlibrary.org/authors/OL23919A/J._K._Rowling',
   'name': 'J. K. Rowling'}],
 'identifiers': {'amazon': ['0439785960'],
  'goodreads': ['53178655'],
  'isbn_10': ['0439785960'],
  'isbn_13': ['9780439785969'],
  'oclc': ['70666878', '819153929'],
  'openlibrary': ['OL24280830M']},
 'classifications': {'lc_classifications': ['PZ7.R79835Halc 2005']},
 'publishers': [{'name': 'Scholastic'}],
 'publish_places': [{'name': 'New York, USA'}],
 'publish_date': '2006-09',
 'subjects': [{'name': 'orphans',
   'url': 'https://openlibrary.org/subjects/orphans'},
  {'name': 'foster homes',
   'url': 'https://openlibrary.org/subjects/foster_homes'},
  {'name': 'romans', 'url': 'https://openlibrary.org/subjects/romans'},
  {'name': 'magie', 'url': 'https://openlibrary.org/subjects/magie'},
  {'name

In [347]:
for index, row in books_df.head(15).iterrows():
    if row['cover_url'] is None:
        isbn = row['isbn13']
        print(f"fetching cover from {row['title']}")
        
        book = fetch_book(isbn)
        if book:
            cover_url = book.get('cover', {}).get('large', '')
            books_df.loc[index, 'cover_url'] = cover_url
        else:
            books_df.loc[index, 'cover_url'] = ""

In [340]:
books_df.head(20)

,title,authors,isbn13,cover_url
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,9780439785969,https://covers.openlibrary.org/b/id/9326654-L.jpg
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,9780439358071,https://covers.openlibrary.org/b/id/12025650-L...
2,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,9780439554930,https://covers.openlibrary.org/b/id/7572543-L.jpg
3,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,9780439554893,https://covers.openlibrary.org/b/id/10301720-L...
4,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,9780439655484,https://covers.openlibrary.org/b/id/10580458-L...
5,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling-Mary GrandPré,9780439682589,https://covers.openlibrary.org/b/id/278981-L.jpg
6,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,9780976540601,https://covers.openlibrary.org/b/id/742235-L.jpg
7,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,9780439827607,https://covers.openlibrary.org/b/id/279436-L.jpg
8,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,9780517226957,https://covers.openlibrary.org/b/id/12617870-L...
9,The Ultimate Hitchhiker's Guide to the Galaxy,Douglas Adams,9780345453747,


## Calling the API with multiple ISBNs at a time